<a href="https://colab.research.google.com/github/cristiandarioortegayubro/BDS/blob/main/modulo.04/bds_algoritmos_004_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>


<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20Scikit-learn.png?raw=true">
</p>


 # **<font color="DeepPink">Modelos supervisados: Regresión Logística con `sklearn`</font>**

 ## **<font color="DeepPink">Bibliotecas</font>**

In [1]:
# Operaciones matemáticas y estadísticas
import pandas as pd
import numpy as np

In [2]:
# Visualización
import plotly.express as px
import plotly.graph_objs as go

 ## **<font color="DeepPink">Conjunto de datos</font>**

<p align="justify">
Volveremos a nuestro conjunto de datos de <i>credit scoring</i>. Sin embargo, esta vez intentaremos predecir la calificación del cliente en base a las variables explicativas.
<br>
<br>
En este caso será un problema de clasificación binaria.

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/cristiandarioortegayubro/BDS/main/datasets/credit_card_completo.csv", index_col=0)

In [5]:
df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,STATUS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,1
1,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,0
2,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,0
3,5008812,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,-22464,365243,1,0,0,0,NaN,1.0,0
4,5008815,M,Y,Y,0,270000.0,Working,Higher education,Married,House / apartment,-16872,-769,1,1,1,1,Accountants,2.0,0


<p align="justify">
Este DataFrame contiene la información del solicitante. El conjunto de datos contiene las siguientes variables:


- **ID**: el número del cliente.

- **CODE_GENDER**: género: M (Masculino) / F (Femenino).

- **FLAG_OWN_CAR**: propietario vehículo: Y (Si) / N (No).

- **FLAG_OWN_REALTY**:  propietario inmueble: Y (Si) / N (No).

- **CNT_CHILDREN**: cantidad de hijos.

- **AMT_INCOME_TOTAL**: ingreso anual.

- **NAME_INCOME_TYPE**: tipo de ingreso.

- **NAME_EDUCATION_TYPE**: nivel educativo.

- **NAME_FAMILY_STATUS**: estado civil.

- **NAME_HOUSING_TYPE**: vivienda.

- **DAYS_BIRTH**: días hasta el nacimiento contando desde hoy hacia atrás, por ejemplo -1 significa ayer.

- **DAYS_EMPLOYED**: días empleado contando desde hoy hacia atrás. Si es positivo, significa la cantidad de días desempleado.

- **FLAG_MOBIL**: tiene número de celular: 1 (Si) / 0 (No).

- **FLAG_WORK_PHONE**: tiene número de teléfono laboral: 1 (Si) / 0 (No).

- **FLAG_PHONE**: tiene número de teléfono fijo: 1 (Si) / 0 (No).

- **FLAG_EMAIL**: tiene dirección de correo electrónico: 1 (Si) / 0 (No).

- **OCCUPATION_TYPE**:	ocupación.

- **CNT_FAM_MEMBERS**:	tamaño del grupo familiar.

- **STATUS**: calificación crediticia: 1 (Mal pagador) / 0 (Buen pagador).

In [6]:
df.shape

(9709, 19)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9709 entries, 0 to 9708
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   9709 non-null   int64  
 1   CODE_GENDER          9709 non-null   object 
 2   FLAG_OWN_CAR         9709 non-null   object 
 3   FLAG_OWN_REALTY      9709 non-null   object 
 4   CNT_CHILDREN         9709 non-null   int64  
 5   AMT_INCOME_TOTAL     9709 non-null   float64
 6   NAME_INCOME_TYPE     9709 non-null   object 
 7   NAME_EDUCATION_TYPE  9709 non-null   object 
 8   NAME_FAMILY_STATUS   9709 non-null   object 
 9   NAME_HOUSING_TYPE    9709 non-null   object 
 10  DAYS_BIRTH           9709 non-null   int64  
 11  DAYS_EMPLOYED        9709 non-null   int64  
 12  FLAG_MOBIL           9709 non-null   int64  
 13  FLAG_WORK_PHONE      9709 non-null   int64  
 14  FLAG_PHONE           9709 non-null   int64  
 15  FLAG_EMAIL           9709 non-null   i

## **<font color="DeepPink">Preprocesamiento de los datos con `pandas`</font>**

<p align="justify">
Se renombran las columnas para asignarles nombres más descriptivos y significativos. Se utiliza el método <code>rename()</code>.

In [8]:
df.rename(columns={'CODE_GENDER':'Genero',
                   'FLAG_OWN_CAR':'Auto',
                   'FLAG_OWN_REALTY':'Propiedad',
                   'CNT_CHILDREN':'Hijos',
                   'AMT_INCOME_TOTAL':'Ingreso_anual',
                   'NAME_EDUCATION_TYPE':'Nivel_educativo',
                   'NAME_FAMILY_STATUS':'Estado_civil',
                   'NAME_HOUSING_TYPE':'Vivienda',
                   'FLAG_EMAIL':'Email',
                   'FLAG_MOBIL':'Celular',
                   'DAYS_BIRTH':'Dias_nacimiento',
                   'DAYS_EMPLOYED':'Dias_empleado',
                   'NAME_INCOME_TYPE':'Tipo_trabajo',
                   'FLAG_WORK_PHONE':'Telefono_laboral',
                   'FLAG_PHONE':'Telefono_fijo',
                   'CNT_FAM_MEMBERS':'Tamaño_familia',
                   'OCCUPATION_TYPE':'Ocupacion',
                   'STATUS':'Calificacion'},
          inplace=True)
df.head(2)

,ID,Genero,Auto,Propiedad,Hijos,Ingreso_anual,Tipo_trabajo,Nivel_educativo,Estado_civil,Vivienda,Dias_nacimiento,Dias_empleado,Celular,Telefono_laboral,Telefono_fijo,Email,Ocupacion,Tamaño_familia,Calificacion
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,1
1,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,0


<p align="justify">
Con el método <code>drop()</code> de <code>Pandas</code> se eliminan las columnas del <code>DataFrame</code> que no son útiles. Descartamos <code>ID</code>, <code>Celular</code> (por tener una sola clase) y <code>Ocupacion</code> (por tener una gran cantidad de valores faltantes).

In [9]:
df.drop(columns = ['ID','Celular','Ocupacion'], inplace=True)

<p align="justify">
Con el método <code>apply()</code> de pandas se puede aplicar una función (<code>lambda</code>) a lo largo de una columna del <code>DataFrame</code> para crear una nueva columna.

In [10]:
df['Situacion_laboral'] = df.Dias_empleado.apply(lambda x: 'Desempleado' if x >= 0 else 'Empleado')
df['Años_empleado'] = df.Dias_empleado.apply(lambda x: round(-x/365.25, 2) if x < 0 else 0)
df['Edad'] = df.Dias_nacimiento.apply(lambda x: round(-x/365.25, 2))
df.drop(columns=["Dias_nacimiento","Dias_empleado"], inplace=True)
df.head()

,Genero,Auto,Propiedad,Hijos,Ingreso_anual,Tipo_trabajo,Nivel_educativo,Estado_civil,Vivienda,Telefono_laboral,Telefono_fijo,Email,Tamaño_familia,Calificacion,Situacion_laboral,Años_empleado,Edad
0,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,1,0,0,2.0,1,Empleado,12.44,32.87
1,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,0,0,0,2.0,0,Empleado,3.10,58.79
2,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,0,1,1,1.0,0,Empleado,8.35,52.32
3,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,0,0,0,1.0,0,Desempleado,0.00,61.50
4,M,Y,Y,0,270000.0,Working,Higher education,Married,House / apartment,1,1,1,2.0,0,Empleado,2.11,46.19


<p align="justify">
El método <code>replace()</code> de pandas se utiliza para reemplazar valores específicos en un <code>DataFrame</code> con nuevos valores. En este caso se usa un <b>diccionario de mapeo</b>.

In [11]:
df.replace({'Telefono_laboral':{1:'Y',0:'N'},
            'Telefono_fijo':{1:'Y',0:'N'},
            'Email':{1:'Y',0:'N'}},
           inplace=True)

<p align="justify">
El método <code>astype()</code> de pandas se utiliza para cambiar el tipo de datos de una columna en un <code>DataFrame</code>.

In [12]:
df.Tamaño_familia = df.Tamaño_familia.astype(int)
df.head()

,Genero,Auto,Propiedad,Hijos,Ingreso_anual,Tipo_trabajo,Nivel_educativo,Estado_civil,Vivienda,Telefono_laboral,Telefono_fijo,Email,Tamaño_familia,Calificacion,Situacion_laboral,Años_empleado,Edad
0,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,Y,N,N,2,1,Empleado,12.44,32.87
1,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,N,N,N,2,0,Empleado,3.10,58.79
2,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,N,Y,Y,1,0,Empleado,8.35,52.32
3,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,N,N,N,1,0,Desempleado,0.00,61.50
4,M,Y,Y,0,270000.0,Working,Higher education,Married,House / apartment,Y,Y,Y,2,0,Empleado,2.11,46.19


 ## **<font color="DeepPink">División del conjunto de datos</font>**

<p align="justify">
👀 Separamos la variable objetivo y las variables predictoras o explicativas.
</p>


In [13]:
X = df.drop(columns='Calificacion')
y = df.Calificacion

<p align="justify">
La función <code>make_column_selector</code> del módulo <code>compose</code> de <code>scikit_learn</code> permite seleccionar columnas específicas de un conjunto de datos.

In [14]:
from sklearn.compose import make_column_selector as selector
numerical_columns_selector = selector(dtype_exclude=object)
numerical_columns = numerical_columns_selector(X)
X_numerical = df[numerical_columns]
X_numerical.head()

,Hijos,Ingreso_anual,Tamaño_familia,Años_empleado,Edad
0,0,427500.0,2,12.44,32.87
1,0,112500.0,2,3.10,58.79
2,0,270000.0,1,8.35,52.32
3,0,283500.0,1,0.00,61.50
4,0,270000.0,2,2.11,46.19


In [15]:
categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(X)
X_categorical = df[categorical_columns]
X_categorical.head()

,Genero,Auto,Propiedad,Tipo_trabajo,Nivel_educativo,Estado_civil,Vivienda,Telefono_laboral,Telefono_fijo,Email,Situacion_laboral
0,M,Y,Y,Working,Higher education,Civil marriage,Rented apartment,Y,N,N,Empleado
1,M,Y,Y,Working,Secondary / secondary special,Married,House / apartment,N,N,N,Empleado
2,F,N,Y,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,N,Y,Y,Empleado
3,F,N,Y,Pensioner,Higher education,Separated,House / apartment,N,N,N,Desempleado
4,M,Y,Y,Working,Higher education,Married,House / apartment,Y,Y,Y,Empleado


 ## **<font color="DeepPink">Codificación de variables categóricas</font>**

<p align="justify">
La clase <code>OneHotEncoder</code> de la biblioteca <code>scikit_learn</code> se utiliza para transformar variables categóricas en representaciones numéricas binarias que pueden ser utilizadas por algoritmos de aprendizaje automático los cuales requieren datos numéricos.

In [16]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output = False)
data_encoded = encoder.fit_transform(X_categorical)
columns_encoded = encoder.get_feature_names_out(X_categorical.columns)
X_categorical_encoded = pd.DataFrame(data_encoded, columns=columns_encoded)
X_categorical_encoded.head()

,Genero_F,Genero_M,Auto_N,Auto_Y,Propiedad_N,Propiedad_Y,Tipo_trabajo_Commercial associate,Tipo_trabajo_Pensioner,Tipo_trabajo_State servant,Tipo_trabajo_Student,...,Vivienda_Rented apartment,Vivienda_With parents,Telefono_laboral_N,Telefono_laboral_Y,Telefono_fijo_N,Telefono_fijo_Y,Email_N,Email_Y,Situacion_laboral_Desempleado,Situacion_laboral_Empleado
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
3,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


 ## **<font color="DeepPink">Normalización de variables numéricas</font>**

<p align="justify">
La clase <code>StandardScaler</code> de la biblioteca <code>scikit_learn</code> en Python se utiliza para estandarizar variables numéricas en un conjunto de datos.

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_numerical_scaled = scaler.fit_transform(X_numerical)
X_numerical_scaled = pd.DataFrame(X_numerical_scaled, columns=X_numerical.columns)
X_numerical_scaled.head()

,Hijos,Ingreso_anual,Tamaño_familia,Años_empleado,Edad
0,-0.551258,2.480773,-0.195755,1.068373,-0.938777
1,-0.551258,-0.692321,-0.195755,-0.404399,1.290914
2,-0.551258,0.894226,-1.267716,0.423444,0.734351
3,-0.551258,1.030216,-1.267716,-0.893221,1.524033
4,-0.551258,0.894226,-0.195755,-0.560507,0.207036


In [18]:
X_numerical_scaled.describe().round(2)

,Hijos,Ingreso_anual,Tamaño_familia,Años_empleado,Edad
count,9709.00,9709.00,9709.00,9709.00,9709.00
mean,0.00,-0.00,-0.00,0.00,-0.00
std,1.00,1.00,1.00,1.00,1.00
min,-0.55,-1.55,-1.27,-0.89,-2.00
25%,-0.55,-0.69,-0.20,-0.75,-0.84
50%,-0.55,-0.24,-0.20,-0.30,-0.09
75%,0.75,0.44,0.88,0.40,0.84
max,24.22,14.04,19.10,5.89,2.16


 ## **<font color="DeepPink">Combinación de Dataframes</font>**

<p align="justify">
En <code>pandas</code>, el método <code>concat()</code> se utiliza para concatenar objetos a lo largo de un eje específico. El parámetro <code>axis</code> se utiliza para indicar a lo largo de qué eje se realizará la concatenación.
<br>
<br>
Cuando se utiliza <code>axis=1</code> en el método <code>concat</code>, se realiza la concatenación a lo largo del eje de las columnas. Esto significa que los objetos se concatenarán uno al lado del otro, creando un nuevo objeto con un mayor número de columnas.

In [19]:
X_transformed = pd.concat([X_categorical_encoded, X_numerical_scaled], axis=1)
X_transformed.head()

,Genero_F,Genero_M,Auto_N,Auto_Y,Propiedad_N,Propiedad_Y,Tipo_trabajo_Commercial associate,Tipo_trabajo_Pensioner,Tipo_trabajo_State servant,Tipo_trabajo_Student,...,Telefono_fijo_Y,Email_N,Email_Y,Situacion_laboral_Desempleado,Situacion_laboral_Empleado,Hijos,Ingreso_anual,Tamaño_familia,Años_empleado,Edad
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,-0.551258,2.480773,-0.195755,1.068373,-0.938777
1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,-0.551258,-0.692321,-0.195755,-0.404399,1.290914
2,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,-0.551258,0.894226,-1.267716,0.423444,0.734351
3,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,-0.551258,1.030216,-1.267716,-0.893221,1.524033
4,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,-0.551258,0.894226,-0.195755,-0.560507,0.207036


<p align="justify">
A continuación, se describe el proceso interno que implica la construcción manual de un pipeline.
<br>
<br>
Este pipeline se compone de una secuencia de pasos que se realizan con el fin de transformar los datos de manera sistemática y eficiente. En primer lugar, se realiza la selección de columnas numéricas y categóricas, con el propósito de distinguir y tratar adecuadamente los diferentes tipos de variables presentes en el conjunto de datos.
<br>
<br>
Posteriormente, se lleva a cabo la codificación de las variables categóricas utilizando el método de codificación <code>one-hot</code>, el cual permite convertir estas variables en una representación binaria. Este proceso es esencial para asegurar que los algoritmos de aprendizaje automático puedan trabajar correctamente con estas variables.
<br>
<br>
Por otro lado, se realiza el escalado de las variables numéricas mediante el uso de la técnica de estandarización. Esta técnica se encarga de ajustar las variables numéricas de manera que tengan una media de cero y una desviación estándar de uno, lo cual es beneficioso para mejorar el rendimiento de algunos algoritmos de aprendizaje automático.
<br>
<br>
Finalmente, se lleva a cabo la concatenación de las columnas generadas por ambas transformaciones, obteniendo así un único <code>DataFrame</code> transformado que encapsula los cambios realizados. Este enfoque de construcción manual del pipeline asegura que cada paso se realice de manera explícita y controlada, permitiendo una mayor comprensión del proceso y ajuste personalizado de las transformaciones aplicadas a los datos.

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/ColumnTransformers-001.png?raw=true" width="500">
</p>


 ## **<font color="DeepPink">División del conjunto de entrenamiento y prueba</font>**

<p align="justify">
Mediante la función <code>train_test_split()</code> del módulo <code>model_selection</code> de <code>scikit_learn</code> se procede a la división del conjunto de datos en conjuntos separados de entrenamiento y prueba.
<br>
<br>
Esta técnica es comúnmente utilizada en aprendizaje automático para evaluar el rendimiento del modelo y verificar su capacidad de generalización a través de datos no vistos previamente.

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_transformed,
                                                    y,
                                                    random_state=123)

 ## **<font color="DeepPink">Ajuste y evaluación del modelo con `sklearn`</font>**

<p align="justify">
En este ejemplo, se crea una instancia de regresión logística mediante la clase <code>LogisticRegression()</code>. Luego, se ajusta el modelo a los datos de entrenamiento utilizando el método <code>fit()</code>, donde <code>X_train</code> representa las variabres predictoras de entrenamiento e <code>y_train</code> es la variable objetivo de entrenamiento correspondiente (<code>STATUS</code>). Una vez ajustado el modelo, se pueden realizar predicciones en los datos de prueba utilizando el método <code>predict()</code>.

In [21]:
from sklearn.linear_model import LogisticRegression
model_1 = LogisticRegression(max_iter=500).fit(X_train, y_train)

In [22]:
prediction = model_1.predict(X_test)
prediction[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [23]:
tabla = pd.DataFrame({"Prediccion":prediction,
                      "Real":y_test,
                      })
tabla.head()

,Prediccion,Real
3998,0,1
8898,0,0
6464,0,0
276,0,0
5243,0,0


<p align="justify">
La clase <code>LogisticRegression</code> también proporciona métodos para la evaluación del modelo, como <code>score()</code> para calcular la <b>accuracy</b> del modelo en los <b>datos de prueba</b>.

In [24]:
model_1.score(X_test, y_test).round(3)

0.868

La **exactitud** o **accuracy** del modelo es 0.868.

 # **<font color="DeepPink">Machine Learning Pipeline</font>**

In [25]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

<p align="justify">
Se ha utilizado <code>categorical_preprocessor</code> como preprocesador o transformador <code>OneHotEncoder</code> con la opción <code>handle_unknown="ignore"</code>, lo cual permite ignorar las clases desconocidas durante la codificación.
<br>
<br>
Es decir, si se encuentra una clase desconocida durante la transformación en los datos de prueba, se ignorará esa clase y se asignará una fila de ceros.
<br>
<br>
Además, se ha utilizado <code>numerical_preprocessor</code> como el preprocesador <code>StandardScaler</code>.

In [26]:
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

<p align="justify">
La clase <code>ColumnTransformer</code> de <code>scikit_learn</code> permite aplicar transformaciones específicas a columnas seleccionadas de un conjunto de datos.
<br>
<br>
<code>ColumnTransformer</code> aplica <code>OneHotEncoder</code> a las columnas categóricas seleccionadas (<code>categorical_columns</code>) y <code>StandardScaler</code> a las columnas numéricas seleccionadas (<code>numerical_columns</code>).
<br>
<br>
El <code>ColumnTransformer</code> toma una <code>lista</code> de <code>tuplas</code>, donde cada <code>tupla</code> representa una transformación que se aplicará a un subconjunto de columnas.
<br>
<br>
Cada tupla tiene tres elementos:


1. El primer elemento de la tupla es un `string` que sirve como nombre descriptivo para la transformación (`'one-hot-encoder'` y `'standard_scaler'`).

2. El segundo elemento de la tupla es el preprocesador o transformador que se utiliza para esa transformación en particular (`categorical_preprocessor` y `numerical_preprocessor`).

3. El tercer elemento de la tupla es una `lista` de nombres de columnas en las que deseas aplicar esa transformación (`categorical_columns` y `numerical_columns`).

In [27]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard_scaler', numerical_preprocessor, numerical_columns)])

<p align="justify">
Por último, se crea un pipeline que incluye <code>preprocessor</code> y el modelo <code>LogisticRegression</code>, el parámetro <code>max_iter</code> indica el número máximo de iteraciones permitidas durante el entrenamiento del modelo de regresión logística.
<br>
<br>
En otras palabras, <code>max_iter</code> limita la cantidad de veces que el algoritmo puede iterar para ajustar el modelo. Cada iteración actualiza los coeficientes del modelo en base a una función de pérdida que busca mejorar la calidad del ajuste.

In [28]:
model_2 = make_pipeline(preprocessor, LogisticRegression(max_iter=500))
model_2

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Genero', 'Auto',
                                                   'Propiedad', 'Tipo_trabajo',
                                                   'Nivel_educativo',
                                                   'Estado_civil', 'Vivienda',
                                                   'Telefono_laboral',
                                                   'Telefono_fijo', 'Email',
                                                   'Situacion_laboral']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['Hijos', 'Ingreso_anual',
                                                   'Tamaño_familia',
                                                   'Años_empleado',
                                                   'Edad'])])),
                ('logisticregression', LogisticRegression(max_iter=500))])

 ## **<font color="DeepPink">División del conjunto de entrenamiento y prueba (*train - test split*)</font>**

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=123)

<p align="justify">
 Se ajusta el modelo con los datos de entrenamiento para hacer predicciones con los datos de prueba.

In [30]:
model_2.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Genero', 'Auto',
                                                   'Propiedad', 'Tipo_trabajo',
                                                   'Nivel_educativo',
                                                   'Estado_civil', 'Vivienda',
                                                   'Telefono_laboral',
                                                   'Telefono_fijo', 'Email',
                                                   'Situacion_laboral']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['Hijos', 'Ingreso_anual',
                                                   'Tamaño_familia',
                                                   'Años_empleado',
                                                   'Edad'])])),
                ('logisticregression', LogisticRegression(max_iter=500))])

In [31]:
model_2.predict(X_test)[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [32]:
model_2.score(X_test, y_test).round(3)

0.868

La **exactitud** o **accuracy** del modelo es 0.868.

 ## **<font color="DeepPink">Evaluación del modelo con *cross-validation*</font>**

In [33]:
from sklearn.model_selection import cross_validate

In [34]:
cv_results = cross_validate(model_2, X, y, cv=5)
cv_results

{'fit_time': array([0.23019981, 0.29629588, 0.21443844, 0.27846003, 0.22941208]),
 'score_time': array([0.01841378, 0.01785636, 0.01807642, 0.018821  , 0.02192616]),
 'test_score': array([0.8676622 , 0.8676622 , 0.8676622 , 0.86817714, 0.86810922])}

In [35]:
scores = cv_results["test_score"]
print("")
print("La accuracy mediante cross-validation es: "
      f"{scores.mean():.3f} ± {scores.std():.5f}")


La accuracy mediante cross-validation es: 0.868 ± 0.00024


 # **<font color="DeepPink">Conclusiones</font>**

<p align="justify">
👀 En este colab nosotros:
<br><br>
✅ Realizamos la construcción manual de un pipeline
<br>
✅ Utilizamos la biblioteca <code>scikit_learn</code> para entrenar un modelo de regresión logística.
<br>
✅ Implementamos un pipeline y aplicamos la validación cruzada. Este pipeline incluyó pasos de preprocesamiento como la codificación de variables categóricas mediante <code>OneHotEncoder</code> y el escalado de variables numéricas mediante <code>StandardScaler</code>.  


<br>
<br>
<p align="center"><b>
💗
<font color="DeepPink">
Hemos llegado al final de nuestro colab, a seguir codeando...
</font>
</p>
<br>
<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>

---
